In [20]:
import pandas as pd
import numpy as np

## 2022

In [21]:
## ruta
p1 = "C:/Users/ivan.corredor/pobreza_calculos_2019_2022/2023/multidimensional/"

In [22]:
#base 2021
#víctimas nal
ipm_22_v = pd.read_csv(p1 + "factores_victima_2022.csv")
# viv nal
ipm_22_viv = pd.read_csv(p1 + "viviendas_nal_2022.csv")
# hog nal
ipm_22_h = pd.read_csv(p1 + "hogares_nal_2022.csv")
# per nal
ipm_22_p = pd.read_csv(p1 + "personas_nal_2022.csv")

### 1. Hogares - viviendas. 2. personas - hog_viv. 3. hog_viv_per - victimas

In [23]:
##volver a mayusculas las variables
ipm_22_viv.columns = ipm_22_viv.columns.str.upper()

###cruce personas - víctimas
per_vic =  pd.merge(left=ipm_22_p, right=ipm_22_v,
                           left_on=['DIRECTORIO',  'SECUENCIA_P', 'ORDEN'], 
                        right_on=['DIRECTORIO', 'SECUENCIA_P', 'ORDEN'],
                           how='left',
                           sort=False, validate='1:1', 
                        suffixes=("_p", "_vic"), indicator = 'indicator_column')

### agrupar personas víctimas por hogar

per_vic = per_vic.assign(VIC2 = per_vic.groupby(['DIRECTORIO', 'SECUENCIA_P'])['VICTIMA'].transform(lambda x: 1 if (x.sum() >= 1) else 0),
                         DESPLA2 = per_vic.groupby(['DIRECTORIO', 'SECUENCIA_P'])['DESPLAZADO'].transform(lambda x: 1 if (x.sum() >= 1) else 0),
                         FEXHOGVIC = per_vic.groupby(['DIRECTORIO', 'SECUENCIA_P'])['FEX_VIC'].transform('mean'))


h_per_vic = pd.merge(left=ipm_22_h, right=per_vic,
                     left_on=['DIRECTORIO', "SECUENCIA_ENCUESTA"], 
                     right_on=['DIRECTORIO', "SECUENCIA_P"], how='inner', 
                     sort=False, validate='1:m', 
                      suffixes=("_h", "_pv"), indicator = 'indicator_column_2')

ipm_22_vhper_vic = pd.merge(left=h_per_vic, right=ipm_22_viv, 
                            left_on=['DIRECTORIO'], 
                            right_on=['DIRECTORIO'], 
                            how='inner',
                            sort=False, 
                            validate='m:1', 
                            suffixes=("_hpv", "_viv"), 
                            indicator = 'indicator_column_3')

### Limpieza, selección y standarización

In [24]:
pd.options.display.max_columns = None
#ipm_21_vhper_vic[["directorio", "ipm", "fex_c_victi"]][ipm_21_vhper_vic["ipm"].notnull()].head(10)

In [25]:
##renombrar variables
ipm_22_vhper_vic.rename(columns = {"p6020":"sexo", "p6040":"edad"}, inplace = True)
#crear variable de año 
ipm_22_vhper_vic["periodo"] = 2022

In [26]:
##crear variable no víctima
def novic(df):
    if df["VIC2"]!=1:
        x = 1
    else:
        x= 0
    return(x)

ipm_22_vhper_vic["no_victima"] = ipm_22_vhper_vic.apply(novic, axis = 1)

In [27]:
#export
#ipm_22_vhper_vic.to_csv(p1 + "Victimas_IPM_2022.csv", sep=";")

### Validaciones 

In [28]:
## número de víctimas muestrales y expandidas
print(ipm_22_vhper_vic["VIC2"].value_counts())
print((ipm_22_vhper_vic["DESPLA2"] * ipm_22_vhper_vic["FEXHOGVIC"]).sum())
print((ipm_22_vhper_vic["pobre"] * ipm_22_vhper_vic["FEXHOGVIC"]).sum())

VIC2
0    161362
1     67526
Name: count, dtype: int64
13511549.086377082
3053664.6054707672


##### IPM expandido población nacional

In [29]:
print(
    round(
    ((ipm_22_vhper_vic["pobre"]* ipm_22_vhper_vic["FEX_C"]).sum() / 
     (ipm_22_vhper_vic["FEX_C"]).sum()*100),2))

12.92


##### IPM expandido población víctima

In [30]:
print(
    round(
    ((ipm_22_vhper_vic["pobre"]* ipm_22_vhper_vic["FEXHOGVIC"]).sum() /
      (ipm_22_vhper_vic["VIC2"] * ipm_22_vhper_vic["FEXHOGVIC"]).sum()*100),1))

20.5


##### IPM expandido población víctima desplazamiento por dominio

In [31]:
### IPM víctimas por dominio
mask_1 = ipm_22_vhper_vic["VIC2"]==1
vic_1 = ipm_22_vhper_vic[mask_1==True]
mask_1 = vic_1["P3"]== 1
vic_1 = vic_1[mask_1==True]

print(
    round(
    ((vic_1["pobre"]* vic_1["FEXHOGVIC"]).sum() /
      (vic_1["VIC2"] * vic_1["FEXHOGVIC"]).sum()*100),1))

14.7


##### IPM expandido población víctima desplazamiento

In [32]:
mask_1 = ipm_22_vhper_vic["DESPLA2"]==1
vic_1 = ipm_22_vhper_vic[mask_1==True]

print(
    round(
    ((vic_1["pobre"]* vic_1["FEXHOGVIC"]).sum() /
      (vic_1["DESPLA2"] * vic_1["FEXHOGVIC"]).sum()*100),3))

21.452


##### IPM expandido población no víctima

In [33]:
print(
    round(
    ((ipm_22_vhper_vic["pobre"]* ipm_22_vhper_vic["FEX_C"]).sum() /
      (ipm_22_vhper_vic["no_victima"] * ipm_22_vhper_vic["FEX_C"]).sum()*100),2))

16.8


## 2021

In [34]:
#base 2021
#víctimas nal
ipm_21_v = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2021/factores_victima_2021")
# viv nal
ipm_21_viv = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2021/viviendas_nal_2021.csv")
# hog nal
ipm_21_h = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2021/hogares_nal_2021.csv")
# per nal
ipm_21_p = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2021/personas_nal_2021.csv")

### 1.  Pegar viviendas a hogares. 2. víctimas de muchos a 1 a hog_viv y luego a la personas 

In [35]:
#Cruce vivienda hogares
ipm_21_vh_merge = pd.merge(left=ipm_21_h, right=ipm_21_viv, left_on=['DIRECTORIO', 'SECUENCIA_ENCUESTA'], 
                        right_on=['DIRECTORIO', 'SECUENCIA_ENCUESTA'], how='left', sort=False, validate='m:1', 
                        suffixes=("_h", "_viv"), indicator = 'indicator_column')

In [36]:
#cruce vivienda_hogares con personas
ipm_21_vhper = pd.merge(left=ipm_21_p, right=ipm_21_vh_merge, left_on=['DIRECTORIO', "SECUENCIA_P"], 
                        right_on=['DIRECTORIO', "SECUENCIA_ENCUESTA"], how='left', sort=False, validate='m:1', 
                        suffixes=("_per", "_ph"), indicator = 'indicator_column_2')

In [37]:
#cruce vivienda_hogares_personas con víctimas
ipm_21_vhper_vic = pd.merge(left=ipm_21_vhper, right=ipm_21_v, left_on=['DIRECTORIO', "SECUENCIA_P", 'ORDEN'], 
                        right_on=['directorio', 'secuencia_p','orden'], how='left',
                            sort=False, validate='1:1', suffixes=("_vhper", "_vic"), indicator = 'indicator_column_3')

### Limpieza, selección y standarización

In [38]:
pd.options.display.max_columns = None
#ipm_21_vhper_vic[["directorio", "ipm", "fex_c_victi"]][ipm_21_vhper_vic["ipm"].notnull()].head(10)

In [39]:
##rellenar variable víctima
def vic(df):
    if df["es_victima"]==1:
        x = 1
    else:
        x= 0
    return(x)

In [40]:
##crear variable no víctima
ipm_21_vhper_vic["victima"] = ipm_21_vhper_vic.apply(vic, axis = 1)

In [41]:
##crear variable no víctima
def novic(df):
    if df["es_victima"]!=1:
        x = 1
    else:
        x= 0
    return(x)

In [42]:
##crear variable no víctima
ipm_21_vhper_vic["no_victima"] = ipm_21_vhper_vic.apply(novic, axis = 1)

In [43]:
##renombrar variables
ipm_21_vhper_vic.rename(columns = {"p6020":"sexo", "p6040":"edad"}, inplace = True)

In [44]:
#crear variable de año 
ipm_21_vhper_vic["periodo"] = 2021

In [45]:
#export
#ipm_21_vhper_vic.to_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2021/Victimas_IPM_2021.csv", sep=";")

### Validaciones 

In [46]:
## número de víctimas muestrales y expandidas
print(ipm_21_vhper_vic["no_victima"].value_counts())
print((ipm_21_vhper_vic["no_victima"] * ipm_21_vhper_vic["fexp"]).sum())
print((ipm_21_vhper_vic["pobre"] * ipm_21_vhper_vic["fexp"]).sum())

no_victima
1    192868
0     41128
Name: count, dtype: int64
167578165.23853242
38986448.10338339


##### IPM expandido población nacional

In [47]:
print(
    round(
    ((ipm_21_vhper_vic["pobre"]* ipm_21_vhper_vic["FEX_C"]).sum() / 
     (ipm_21_vhper_vic["FEX_C"]).sum()*100),2))

15.97


##### IPM expandido población víctima

In [48]:
print(
    round(
    ((ipm_21_vhper_vic["pobre"]* ipm_21_vhper_vic["fex_c_victi"]).sum() /
      (ipm_21_vhper_vic["victima"] * ipm_21_vhper_vic["fex_c_victi"]).sum()*100),2))

24.41


##### IPM expandido población víctima desplazamiento

In [49]:
print(
    round(
    ((ipm_21_vhper_vic["pobre"]* ipm_21_vhper_vic["fex_c_des"]).sum() /
      (ipm_21_vhper_vic["es_desplazado"] * ipm_21_vhper_vic["fex_c_des"]).sum()*100),2))

25.03


##### IPM expandido población no víctima

In [50]:
print(
    round(
    ((ipm_21_vhper_vic["pobre"]* ipm_21_vhper_vic["FEX_C"]).sum() /
      (ipm_21_vhper_vic["no_victima"] * ipm_21_vhper_vic["FEX_C"]).sum()*100),2))

18.33


## 2020

In [51]:
#base 2020
#víctimas nal
ipm_20_v = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2020/factores_victima_2020")
# viv nal
ipm_20_viv = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2020/viviendas_nal_2020.csv", sep=";", decimal= ",")
# hog nal
ipm_20_h = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2020/hogares_nal_2020.csv", sep=";", decimal= ",")
# per nal
ipm_20_p = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2020/personas_nal_2020.csv", sep=";", decimal= ",")

In [52]:
#Cruce vivienda hogares
ipm_20_vh_merge = pd.merge(left=ipm_20_h, right=ipm_20_viv, left_on=['DIRECTORIO', 'SECUENCIA_ENCUESTA'], 
                        right_on=['DIRECTORIO', 'SECUENCIA_ENCUESTA'], how='left', sort=False, validate='m:1', 
                        suffixes=("_h", "_viv"), indicator = 'indicator_column')

In [53]:
#cruce vivienda_hogares con personas
ipm_20_vhper = pd.merge(left=ipm_20_p, right=ipm_20_vh_merge, left_on=['DIRECTORIO', "SECUENCIA_P"], 
                        right_on=['DIRECTORIO', "SECUENCIA_ENCUESTA"], how='left', sort=False, validate='m:1', 
                        suffixes=("_per", "_ph"), indicator = 'indicator_column_2')

In [54]:
#cruce vivienda_hogares_personas con víctimas
ipm_20_vhper_vic = pd.merge(left=ipm_20_vhper, right=ipm_20_v, left_on=['DIRECTORIO', "SECUENCIA_P",
                                                                        'ORDEN'], 
                        right_on=['directorio',  'secuencia_p','orden'], how='left',
                            sort=False, validate='1:1', suffixes=("_vhper", "_vic"), indicator = 'indicator_column_3')

In [55]:
pd.options.display.max_columns = None

In [56]:
##rellenar variable víctima
def vic(df):
    if df["es_victima"]==1:
        x = 1
    else:
        x= 0
    return(x)

In [57]:
##crear variable no víctima
ipm_20_vhper_vic["victima"] = ipm_20_vhper_vic.apply(vic, axis = 1)

In [58]:
##crear variable no víctima
def novic(df):
    if df["es_victima"]!=1:
        x = 1
    else:
        x= 0
    return(x)

In [59]:
##crear variable no víctima
ipm_20_vhper_vic["no_victima"] = ipm_20_vhper_vic.apply(novic, axis = 1)

In [60]:
##renombrar variables
ipm_20_vhper_vic.rename(columns = {"p6020":"sexo", "p6040":"edad"}, inplace = True)

In [61]:
#crear variable de año 
ipm_20_vhper_vic["periodo"] = 2020

In [62]:
#export
#ipm_20_vhper_vic.to_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2020/Victimas_IPM_2020.csv", sep=";")

##### IPM expandido población nacional

In [63]:
print(
    round(
    ((ipm_20_vhper_vic["pobre"]* ipm_20_vhper_vic['fex_c_per']).sum() / 
     (ipm_20_vhper_vic['fex_c_per']).sum()*100),2))

18.11


##### IPM expandido población víctima

In [64]:
print(
    round(
    ((ipm_20_vhper_vic["pobre"]* ipm_20_vhper_vic["fex_c_victi"]).sum() /
      (ipm_20_vhper_vic["victima"] * ipm_20_vhper_vic["fex_c_victi"]).sum()*100),1))

30.3


In [65]:
#cabecera
cab_20 = ipm_20_vhper_vic[ipm_20_vhper_vic["P3"].isin([1])]
print(
    round(
    ((cab_20["pobre"]* cab_20['fex_c_victi']).sum() /
      (cab_20['fex_c_victi']).sum()*100),1))

21.5


In [66]:
#centro y rural
c_r_20 = ipm_20_vhper_vic[ipm_20_vhper_vic["P3"].isin([2,3])]
print(
    round(
    ((c_r_20["pobre"]* c_r_20['fex_c_victi']).sum() /
      (c_r_20['fex_c_victi']).sum()*100),1))


45.7


##### IPM expandido población víctima desplazamiento

In [67]:
print(
    round(
    ((ipm_20_vhper_vic["pobre"]* ipm_20_vhper_vic['fex_c_des']).sum() /
      (ipm_20_vhper_vic['es_desplazado'] * ipm_20_vhper_vic['fex_c_des']).sum()*100),1))

31.3


In [68]:
#centro y rural
print(
    round(
    ((c_r_20["pobre"]* c_r_20['fex_c_des']).sum() /
      (c_r_20['fex_c_des']).sum()*100),1))

46.3


In [69]:
print(
    round(
    ((cab_20["pobre"]* cab_20['fex_c_des']).sum() /
      (cab_20['fex_c_des']).sum()*100),1))

22.4


##### IPM expandido población no víctima

In [70]:
print(
    round(
    ((ipm_20_vhper_vic["pobre"]* ipm_20_vhper_vic["fex_c_per"]).sum() /
      (ipm_20_vhper_vic["no_victima"] * ipm_20_vhper_vic["fex_c_per"]).sum()*100),2))

20.71


## 2019

In [71]:
#base 1919
#víctimas nal
ipm_19_v = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2019/factores_victima_2019")
# viv nal
ipm_19_viv = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2019/viviendas_nal_2019.csv", sep=";", decimal= ",")
# hog nal
ipm_19_h = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2019/hogares_nal_2019.csv", sep=";", decimal= ",")
# per nal
ipm_19_p = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2019/personas_nal_2019.csv", sep=";", decimal= ",")

In [72]:
#Cruce vivienda hogares
ipm_19_vh_merge = pd.merge(left=ipm_19_h, right=ipm_19_viv, left_on=['DIRECTORIO', 'SECUENCIA_ENCUESTA'], 
                        right_on=['DIRECTORIO', 'SECUENCIA_ENCUESTA'], how='left', sort=False, validate='m:1', 
                        suffixes=("_h", "_viv"), indicator = 'indicator_column')

In [73]:
#cruce vivienda_hogares con personas
ipm_19_vhper = pd.merge(left=ipm_19_p, right=ipm_19_vh_merge, left_on=['DIRECTORIO', "SECUENCIA_P"], 
                        right_on=['DIRECTORIO', "SECUENCIA_ENCUESTA"], how='left', sort=False, validate='m:1', 
                        suffixes=("_per", "_ph"), indicator = 'indicator_column_2')

In [74]:
#cruce vivienda_hogares_personas con víctimas
ipm_19_vhper_vic = pd.merge(left=ipm_19_vhper, right=ipm_19_v, left_on=['DIRECTORIO', "SECUENCIA_P", 'ORDEN'], 
                        right_on=['directorio', 'secuencia_p','orden'], how='left',
                            sort=False, validate='1:1', suffixes=("_vhper", "_vic"), indicator = 'indicator_column_3')

In [75]:
pd.options.display.max_columns = None

In [76]:
##rellenar variable víctima
def vic(df):
    if df["es_victima"]==1:
        x = 1
    else:
        x= 0
    return(x)

In [77]:
##crear variable no víctima
ipm_19_vhper_vic["victima"] = ipm_19_vhper_vic.apply(vic, axis = 1)

In [78]:
##crear variable no víctima
def novic(df):
    if df["es_victima"]!=1:
        x = 1
    else:
        x= 0
    return(x)

In [79]:
##crear variable no víctima
ipm_19_vhper_vic["no_victima"] = ipm_19_vhper_vic.apply(novic, axis = 1)

In [80]:
##renombrar variables
ipm_19_vhper_vic.rename(columns = {"p6020":"sexo", "p6040":"edad"}, inplace = True)

In [81]:
#crear variable de año 
ipm_19_vhper_vic["periodo"] = 2019

In [82]:
#export
#ipm_20_vhper_vic.to_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/2020/Victimas_IPM_2020.csv", sep=";")

##### IPM expandido población nacional

In [83]:
print(
    round(
    ((ipm_19_vhper_vic["pobre"]* ipm_19_vhper_vic['fex_c_per']).sum() / 
     (ipm_19_vhper_vic['fex_c_per']).sum()*100),2))

17.45


##### IPM expandido población víctima

In [84]:
print(
    round(
    ((ipm_19_vhper_vic["pobre"]* ipm_19_vhper_vic["fex_c_victi"]).sum() /
      (ipm_19_vhper_vic["victima"] * ipm_19_vhper_vic["fex_c_victi"]).sum()*100),2))

26.87


In [85]:
#cabecera
cab_19 = ipm_19_vhper_vic[ipm_19_vhper_vic["P3"].isin([1])]
print(
    round(
    ((cab_19["pobre"]* cab_19['fex_c_victi']).sum() /
      (cab_19['victima'] * cab_19['fex_c_victi']).sum()*100),2))

21.95


In [86]:
#centro y rural
c_r_19 = ipm_19_vhper_vic[ipm_19_vhper_vic["P3"].isin([2,3])]
print(
    round(
    ((c_r_19["pobre"]* c_r_19['fex_c_victi']).sum() /
      (c_r_19['victima']* c_r_19['fex_c_victi']).sum()*100),1))


37.4


##### IPM expandido población víctima desplazamiento

In [87]:
print(
    round(
    ((ipm_19_vhper_vic["pobre"]* ipm_19_vhper_vic['fex_c_des']).sum() /
      (ipm_19_vhper_vic['es_desplazado'] * ipm_19_vhper_vic['fex_c_des']).sum()*100),3))

27.532


##### IPM expandido población no víctima

In [88]:
print(
    round(
    ((ipm_19_vhper_vic["pobre"]* ipm_19_vhper_vic["fex_c_per"]).sum() /
      (ipm_19_vhper_vic["no_victima"] * ipm_19_vhper_vic["fex_c_per"]).sum()*100),2))

19.95


## Nacional 2019, 2020, 2021, 2022

In [94]:
# selección de variables
var19 = list(ipm_19_vhper_vic.columns)

var20 = list(ipm_20_vhper_vic.columns)

var21 = list(ipm_21_vhper_vic.columns)

var22 = list(ipm_22_vhper_vic.columns)

In [90]:
print(len(var19), len(var20), len(var21), len(var22))

76 76 79 84


In [93]:
#renombrar variable
ipm_19_vhper_vic.rename(columns= {'P6390S1':'P6390'}, inplace=True)

#borrar variable parentesco p6051 ya que no viene en la base de personas de 2021
ipm_19_vhper_vic.drop(columns ="P6051", inplace =True)
ipm_20_vhper_vic.drop(columns ="P6051", inplace =True)

## reemplazar nombre de factor 'FEX_C' en base 2021
ipm_21_vhper_vic.rename(columns= {'FEX_C':'fex_c_per', 'fex_c':'fex_c_ph'}, inplace=True)

###BORRAR 4 VARIABLES de 2021 QUE NO SE ENCUENTRAN EN 2019 Y 2020
ipm_21_vhper_vic.drop(columns =[ 'P3336S1', 'P3336S2','P3336S3', 'P3337'], inplace =True)

###BORRAR 4 VARIABLES de 2022 QUE NO SE ENCUENTRAN EN 2019 Y 2020
ipm_22_vhper_vic.drop(columns =[ 'P3336S1', 'P3336S2','P3336S3', 'P3337'], inplace =True)

#renombrar variables de tabla 2022:
ipm_22_vhper_vic.rename(columns= { 'VIC2': 'es_victima', 
                                   'FEX_C':'fex_c_per',
                                   'FEXHOGVIC': 'fex_c_victi',
                                   'DESPLA2': 'es_desplazado',
                                   'VICTIMA': 'victima',
                                   'FEX_VIC': 'fex_c_victi2',
                                   'DESPLAZADO': 'es_desplazado2' 
                                   }, inplace=True)

In [96]:
print(len(var19), len(var20), len(var21), len(var22))

75 75 75 80


In [97]:
#concatenar periodos 19-22
frames = [ipm_19_vhper_vic, ipm_20_vhper_vic, ipm_21_vhper_vic, ipm_22_vhper_vic]
nal_19_22= pd.concat(frames, axis=0)

In [98]:
##idenficar los elementos distintos 19-20
distinct_var19 = set(var19).difference(set(var20))
print("Distinct elements in var19:", distinct_var19)

Distinct elements in var19: set()


In [99]:
##idenficar los elementos distintos 19-21
distinct_var19 = set(var19).difference(set(var21))
print("Distinct elements in var19:", distinct_var19)

Distinct elements in var19: set()


In [605]:
##idenficar los elementos distintos 20-21
distinct_var20 = set(var20).difference(set(var21))
print("Distinct elements in var20:", distinct_var20)

Distinct elements in var20: {'fex_c_per', 'P6051', 'fex_c_ph'}


### Alistamiento base 2022

In [95]:
##idenficar los elementos distintos 19-21
print("Distinct elements in var19 - var22:", set(var22).difference(set(var19)))

Distinct elements in var19 - var22: {'SECUENCIA_P_pv', 'SECUENCIA_ENCUESTA_hpv', 'P1077S21', 'P1075', 'P6051', 'SECUENCIA_ENCUESTA_viv', 'P1077S23', 'es_desplazado2', 'SECUENCIA_ENCUESTA_vic', 'SECUENCIA_ENCUESTA_p', 'P1082S2', 'fex_c', 'P1077S22', 'fex_c_victi2'}


### validación 19-22

In [100]:
nal_19_22.columns

Index(['DIRECTORIO', 'SECUENCIA_ENCUESTA_per', 'SECUENCIA_P', 'ORDEN', 'P6020',
       'P6040', 'fex_c_per', 'P6090', 'P5665', 'P8563', 'P51', 'P55', 'P774',
       'P6160', 'P8586', 'P8587', 'P8587S1', 'P1088', 'P1088S1', 'P6180',
       'P6240', 'P6250', 'P6260', 'P6270', 'P6351', 'P6390', 'P7250', 'P6920',
       'PERIODO', 'SECUENCIA_ENCUESTA_ph', 'SECUENCIA_P_h', 'P5010', 'P8526',
       'P8530', 'fex_c_ph', 'REGION', 'personas', 'paredes', 'pisos',
       'alcantarillado', 'acueducto', 'empleo_formal',
       'desempleo_larga_duracion', 'barreras_acceso_salud',
       'aseguramiento_salud', 'trabajo_infantil', 'atencion_integral',
       'inasistencia_escolar', 'rezago_escolar', 'alfabetismo',
       'logro_educativo', 'hacinamiento', 'ipm', 'pobre', 'fexp',
       'SECUENCIA_P_viv', 'P3', 'P4005', 'P4015', 'P8520S3', 'P8520S5',
       'indicator_column', 'indicator_column_2', 'directorio',
       'secuencia_encuesta', 'secuencia_p', 'orden', 'es_victima',
       'es_desplazado',

In [103]:
nal_22= nal_19_22[(nal_19_22['periodo']==2022) & (nal_19_22['es_desplazado']==1)]
print(
    round(
    ((nal_22["pobre"]* nal_22['fex_c_victi']).sum() /
      (nal_22['es_desplazado'] * nal_22['fex_c_victi']).sum()*100),2))

21.45


In [134]:
nal_19_22[(nal_19_22['periodo']==2022)]["es_victima"] .value_counts()

es_victima
0.0    161362
1.0     67526
Name: count, dtype: int64

In [135]:
#export
nal_19_22.to_csv(p1+ "nal_19_22.csv", sep=";")

In [ ]:
#nal_19_22 = pd.read_csv(p1 + "nal_19_22.csv", sep=";", low_memory=False)

In [136]:
## etiquetar la variable región
nal_19_22["REGION"] = nal_19_22["REGION"].replace([1,2,3,4,5,6,7,8,9], ["Caribe", "Oriental", "Central", "Pacífica (sin valle)", "Bogotá", 
                                                                       "Antioquia", "Valle del Cauca", 
                                                                        "San Andrés y Providencia", "Orinoquía y Amazonía"])

In [137]:
#exportar nuevamente
nal_19_22.to_csv(p1+ "nal_19_22.csv", sep=";")

In [107]:
##comprobación tasa nacional
nal_22= nal_19_22[(nal_19_22['periodo']==2022) & (nal_19_22['REGION']=="Caribe") & (nal_19_22['es_desplazado']==1)]
print(
    round(
    ((nal_22["pobre"]* nal_22['fex_c_victi']).sum() /
      (nal_22['es_desplazado'] * nal_22['fex_c_victi']).sum()*100),2))

26.52


In [152]:
# ipm por regiones. nacional
reg_nal = nal_19_22.groupby(["periodo", "REGION"]).apply(lambda x: ((x['pobre'] * x['fex_c_per']).sum() / x['fex_c_per'].sum()) * 100)
reg_nal 

periodo  REGION                  
2019     Antioquia                   15.698502
         Bogotá                       7.052955
         Caribe                      28.128625
         Central                     15.213249
         Oriental                    15.218896
         Orinoquía y Amazonía        20.224209
         Pacífica (sin valle)        26.332730
         San Andrés y Providencia     8.170257
         Valle del Cauca             10.794780
2020     Antioquia                   14.877727
         Bogotá                       7.506987
         Caribe                      28.747925
         Central                     17.925735
         Oriental                    14.508147
         Orinoquía y Amazonía        21.469713
         Pacífica (sin valle)        30.888642
         San Andrés y Providencia    11.910315
         Valle del Cauca             11.130451
2021     Antioquia                   14.273028
         Bogotá                       5.676359
         Caribe           

In [109]:
# ipm por regiones. nacional no víctimas
reg_nal_nv = nal_19_22.groupby(["periodo", "REGION"]).apply(
    lambda x: ((x['pobre'] * x['fex_c_per']).sum() / (x['no_victima'] * x['fex_c_per']).sum() * 100))
reg_nal_nv

periodo  REGION                  
2019     Antioquia                   19.348020
         Bogotá                       7.243587
         Caribe                      33.130440
         Central                     17.412757
         Oriental                    16.557472
         Orinoquía y Amazonía        25.577040
         Pacífica (sin valle)        35.259168
         San Andrés y Providencia     8.258068
         Valle del Cauca             12.094639
2020     Antioquia                   17.930120
         Bogotá                       7.775595
         Caribe                      33.962981
         Central                     20.525757
         Oriental                    15.812521
         Orinoquía y Amazonía        27.255708
         Pacífica (sin valle)        42.028958
         San Andrés y Providencia    12.044836
         Valle del Cauca             12.435986
2021     Antioquia                   17.480578
         Bogotá                       5.861561
         Caribe           

In [111]:
# ipm por regiones. Víctimas todos los hechos
reg_vic_tot = nal_19_22.groupby(["periodo", "REGION"]).apply(
    lambda x: ((x['pobre'] * x['fex_c_victi']).sum() / (x['es_victima'] * x['fex_c_victi']).sum() * 100))
reg_vic_tot

periodo  REGION                  
2019     Antioquia                   21.494652
         Bogotá                      12.349230
         Caribe                      33.203663
         Central                     22.929137
         Oriental                    22.278850
         Orinoquía y Amazonía        23.789535
         Pacífica (sin valle)        34.785937
         San Andrés y Providencia     0.000000
         Valle del Cauca             22.795928
2020     Antioquia                   25.445537
         Bogotá                      18.002321
         Caribe                      34.870252
         Central                     28.414497
         Oriental                    23.792685
         Orinoquía y Amazonía        22.887194
         Pacífica (sin valle)        44.165051
         San Andrés y Providencia     0.000000
         Valle del Cauca             19.529599
2021     Antioquia                   19.528957
         Bogotá                      10.475636
         Caribe           

In [112]:
# ipm por regiones. Víctimas desplazamiento
reg_vic_des = nal_19_22[nal_19_22['es_desplazado']==1].groupby(["periodo", "REGION"]).apply(
    lambda x: ((x['pobre'] * x['fex_c_victi']).sum() / (x['es_desplazado'] * x['fex_c_victi']).sum() * 100))
reg_vic_des

periodo  REGION                  
2019     Antioquia                   22.768412
         Bogotá                      11.320394
         Caribe                      33.546396
         Central                     23.209571
         Oriental                    22.546469
         Orinoquía y Amazonía        24.607070
         Pacífica (sin valle)        35.185805
         San Andrés y Providencia     0.000000
         Valle del Cauca             23.508782
2020     Antioquia                   27.082699
         Bogotá                      19.102597
         Caribe                      35.173624
         Central                     29.727878
         Oriental                    24.403565
         Orinoquía y Amazonía        23.349356
         Pacífica (sin valle)        45.347031
         San Andrés y Providencia     0.000000
         Valle del Cauca             20.146365
2021     Antioquia                   20.213834
         Bogotá                      10.748052
         Caribe           

### bases departamentales 2019-2022

In [138]:
#cargue y pegue 19-20
for i in range(19,21):
    exec('vic{} = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/20{}/factores_victima_20{}")'.format(i,i,i))
    exec('viv{} = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/20{}/viviendas_dpto_20{}.csv", sep=";", decimal= ",")'.format(i,i,i))
    exec('h{} = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/20{}/hogares_dpto_20{}.csv", sep=";", decimal= ",")'.format(i,i,i))
    exec('per{} = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/20{}/personas_dpto_20{}.csv", sep=";", decimal= ",")'.format(i,i,i))
    #cruce vivienda_hogares
    exec('dpto_vh{} = pd.merge(left=h{}, right=viv{}, left_on=["DIRECTORIO", "SECUENCIA_ENCUESTA"], right_on=["DIRECTORIO", "SECUENCIA_ENCUESTA"], how="left", sort=False, validate="m:1", suffixes=("_h", "_viv"), indicator = "indicator_column")'.format(i,i,i))
    #cruce vivienda_hogares con personas
    exec('dpto_vhper{} = pd.merge(left=per{}, right=dpto_vh{}, left_on=["DIRECTORIO", "SECUENCIA_P"], right_on=["DIRECTORIO", "SECUENCIA_ENCUESTA"], how="left", sort=False, validate="m:1", suffixes=("_per", "_ph"), indicator = "indicator_column_2")'.format(i,i,i))
    #cruce vivienda_hogares_personas con víctimas
    exec('dpto_vhper_vic{} = pd.merge(left=dpto_vhper{}, right=vic{}, left_on=["DIRECTORIO", "SECUENCIA_P", "ORDEN"], right_on=["directorio", "secuencia_p","orden"], how="left",sort=False, validate="1:1", suffixes=("_vhper", "_vic"), indicator = "indicator_column_3")'.format(i,i,i))

In [139]:
#cargue 21
for i in range(21,22):
    exec('vic{} = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/20{}/factores_victima_20{}")'.format(i,i,i))
    exec('viv{} = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/20{}/viviendas_dpto_20{}.csv")'.format(i,i,i))
    exec('h{} = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/20{}/hogares_dpto_20{}.csv")'.format(i,i,i))
    exec('per{} = pd.read_csv("C:/Users/ivan.corredor/DANE_IPM_ECV/20{}/personas_dpto_20{}.csv")'.format(i,i,i))
    #cruce vivienda_hogares
    exec('dpto_vh{} = pd.merge(left=h{}, right=viv{}, left_on=["DIRECTORIO", "SECUENCIA_ENCUESTA"], right_on=["DIRECTORIO", "SECUENCIA_ENCUESTA"], how="left", sort=False, validate="m:1", suffixes=("_h", "_viv"), indicator = "indicator_column")'.format(i,i,i))
    #cruce vivienda_hogares con personas
    exec('dpto_vhper{} = pd.merge(left=per{}, right=dpto_vh{}, left_on=["DIRECTORIO", "SECUENCIA_P"], right_on=["DIRECTORIO", "SECUENCIA_ENCUESTA"], how="left", sort=False, validate="m:1", suffixes=("_per", "_ph"), indicator = "indicator_column_2")'.format(i,i,i))
    #cruce vivienda_hogares_personas con víctimas
    exec('dpto_vhper_vic{} = pd.merge(left=dpto_vhper{}, right=vic{}, left_on=["DIRECTORIO", "SECUENCIA_P", "ORDEN"], right_on=["directorio", "secuencia_p","orden"], how="left",sort=False, validate="1:1", suffixes=("_vhper", "_vic"), indicator = "indicator_column_3")'.format(i,i,i))

In [140]:
#cargue 22
vic22 = pd.read_csv(p1 + "factores_victima_2022.csv")
viv22 = pd.read_csv(p1 + "viviendas_dpto_2022.csv")
viv22.columns = viv22.columns.str.upper()
h22 = pd.read_csv(p1 + "hogares_dpto_2022.csv")
per22 = pd.read_csv(p1 + "personas_dpto_2022.csv")

In [141]:
###cruce personas - víctimas
dpto_per_vic =  pd.merge(left=per22, right=vic22,
                           left_on=['DIRECTORIO',  'SECUENCIA_P', 'ORDEN'], 
                        right_on=['DIRECTORIO', 'SECUENCIA_P', 'ORDEN'],
                           how='left',
                           sort=False, validate='1:1', 
                        suffixes=("_p", "_vic"), indicator = 'indicator_column')

### agrupar personas víctimas por hogar

dpto_per_vic = per_vic.assign(VIC2 = dpto_per_vic.groupby(['DIRECTORIO', 'SECUENCIA_P'])['VICTIMA'].transform(lambda x: 1 if (x.sum() >= 1) else 0),
                         DESPLA2 = dpto_per_vic.groupby(['DIRECTORIO', 'SECUENCIA_P'])['DESPLAZADO'].transform(lambda x: 1 if (x.sum() >= 1) else 0),
                         FEXHOGVIC = dpto_per_vic.groupby(['DIRECTORIO', 'SECUENCIA_P'])['FEX_VIC'].transform('mean'))

### hogares - personas victimas 
dpto_h_per_vic = pd.merge(left=h22, right = dpto_per_vic,
                     left_on=['DIRECTORIO', "SECUENCIA_ENCUESTA"], 
                     right_on=['DIRECTORIO', "SECUENCIA_P"], how='inner', 
                     sort=False, validate='1:m', 
                      suffixes=("_h", "_pv"), indicator='indicator_column_2')
### hogares personas victimas - vivienda
dpto_vhper_vic22 = pd.merge(left=dpto_h_per_vic, right=viv22, 
                            left_on=['DIRECTORIO'], 
                            right_on=['DIRECTORIO'], 
                            how='inner',
                            sort=False, 
                            validate='m:1', 
                            suffixes=("_hpv", "_viv"), 
                            indicator = 'indicator_column_3')

In [142]:
### renombrar variables del año 2022
dpto_vhper_vic22.rename(columns= { 'VIC2': 'es_victima', 
                                   'FEX_C':'fex_c_per',
                                   'FEXHOGVIC': 'fex_c_victi',
                                   'DESPLA2': 'es_desplazado',
                                   'VICTIMA': 'victima',
                                   'FEX_VIC': 'fex_c_victi2',
                                   'DESPLAZADO': 'es_desplazado2' 
                                   }, inplace=True)

In [143]:
##crear variable no víctima
def novic(df):
    if df["es_victima"]!=1:
        x = 1
    else:
        x= 0
    return(x)

In [144]:
for i in range(19,23):
    ##crear variable víctima
    exec('dpto_vhper_vic{}["victima"] = dpto_vhper_vic{}.apply(vic, axis = 1)'.format(i,i))
    ##crear variable no víctima
    exec('dpto_vhper_vic{}["no_victima"] = dpto_vhper_vic{}.apply(novic, axis = 1)'.format(i,i))
    #crear variable de año 
    exec('dpto_vhper_vic{}["periodo"] = 20{}'.format(i,i))
    #exec('dpto_vhper_vic{}.rename(columns = {"p6020":"sexo", "p6040":"edad"}, inplace = True)'.format(i))

In [145]:
##renombrar variables
dpto_vhper_vic19.rename(columns = {"p6020":"sexo", "p6040":"edad"}, inplace = True)
dpto_vhper_vic20.rename(columns = {"p6020":"sexo", "p6040":"edad"}, inplace = True)
dpto_vhper_vic21.rename(columns = {"p6020":"sexo", "p6040":"edad"}, inplace = True)
dpto_vhper_vic22.rename(columns = {"p6020":"sexo", "p6040":"edad"}, inplace = True)

#### Homogenización de las bases para concatenación

In [146]:
# selección de variables
vard19 = list(dpto_vhper_vic19.columns)
vard20 = list(dpto_vhper_vic20.columns)
vard21 = list(dpto_vhper_vic21.columns)
vard22 = list(dpto_vhper_vic22.columns)
print(len(vard19), len(vard20), len(vard21), len(vard22))

77 77 80 85


In [147]:
#renombrar variable
dpto_vhper_vic19.rename(columns= {'P6390S1':'P6390'}, inplace=True)
dpto_vhper_vic21.rename(columns= {'FEX_C':'fex_c_per', 'fex_c':'fex_c_ph'}, inplace=True)

###BORRAR 4 VARIABLES QUE NO SE ENCUENTRAN EN 2019 Y 2020
dpto_vhper_vic21.drop(columns =['P3336S1', 'P3336S2','P3336S3', 'P3337'], inplace =True)
###BORRAR 4 VARIABLES QUE NO SE ENCUENTRAN EN 2019 Y 2020
dpto_vhper_vic22.drop(columns =['P3336S1', 'P3336S2','P3336S3', 'P3337'], inplace =True)


In [148]:
#concatenar periodos 19-22
frames_dpto = [dpto_vhper_vic19, dpto_vhper_vic20, dpto_vhper_vic21, dpto_vhper_vic22]
dpto_19_22= pd.concat(frames_dpto, axis=0)

In [149]:
##idenficar los elementos distintos 19-20
distinct_vard19 = set(vard19).difference(set(vard20))
print("Distinct elements in vard19:", distinct_vard19)

Distinct elements in vard19: {'P6390S1'}


In [150]:
##idenficar los elementos distintos 19-20
distinct_vard19 = set(vard19).difference(set(vard20))
print("Distinct elements in vard19:", distinct_vard19)

Distinct elements in vard19: {'P6390S1'}


In [151]:
#export
dpto_19_22.to_csv(p1 + "dpto_19_22.csv", sep=";")

### Resultados Departamentos

Esta sección de código se corre en el script pobreza_por_dpto

In [ ]:
pd.options.display.max_rows=None

In [219]:
print(list(sorted(dpto_19_22["DEPARTAMENTO"].unique())))

[5, 8, 11, 13, 15, 17, 18, 19, 20, 23, 25, 27, 41, 44, 47, 50, 52, 54, 63, 66, 68, 70, 73, 76, 81, 85, 86, 88, 91, 94, 95, 97, 99]


In [220]:
## etiquetar la variable región
dpto_19_22["DEPARTAMENTO"] = dpto_19_22["DEPARTAMENTO"].replace([5,8,11,13,15,17,18,19,20,23,25,27,41,44,47,50,52,54,63,66,68,
                                                                70, 73, 76, 81, 85,86, 88, 91,94, 95,97,99],
                                                                ["Antioquia", "Atlántico", "Bogotá", "Bolívar","Boyacá", 
                                                                 "Caldas","Caquetá","Cauca", "Cesar","Córdoba", "Cundinamarca",
                                                                "Chocó", "Huila", "La Guajira", "Magdalena", "Meta", "Nariño",
                                                                "Norte de Santander", "Quindio", "Risaralda", "Santander",
                                                                "Sucre", "Tolima", "Valle del Cauca", "Arauca", "Casanare",
                                                                "Putumayo", "Providencia y Santa Catalina", "Amazonas", 
                                                                "Guainía", "Guaviare", "Vaupés", "Vichada"])

In [221]:
# ipm por regiones. nacional
dpto_nal = dpto_19_22.groupby(["periodo", "DEPARTAMENTO"]).apply(lambda x: ((x['pobre'] * x['fex_c_per']).sum() / x['fex_c_per'].sum()) * 100)
dpto_nal 

periodo  DEPARTAMENTO   
2019     Amazonas           35.647439
         Antioquia          15.698502
         Arauca             23.312620
         Atlántico          14.939818
         Bogotá              7.086484
                              ...    
2022     Sucre              26.015727
         Tolima             10.541457
         Valle del Cauca     9.661405
         Vaupés             47.100273
         Vichada            75.396661
Length: 132, dtype: float64

In [222]:
# ipm por regiones. nacional no víctimas
dpto_nal_nv = dpto_19_22.groupby(["periodo", "DEPARTAMENTO"]).apply(
    lambda x: ((x['pobre'] * x['fex_c_per']).sum() / (x['no_victima'] * x['fex_c_per']).sum() * 100))
dpto_nal_nv

periodo  DEPARTAMENTO   
2019     Amazonas           36.601344
         Antioquia          19.348020
         Arauca             29.763298
         Atlántico          15.919329
         Bogotá              7.278477
                              ...    
2022     Sucre              37.254039
         Tolima             12.002864
         Valle del Cauca    10.757918
         Vaupés             55.941077
         Vichada            80.444868
Length: 132, dtype: float64

In [223]:
# ipm por regiones. Víctimas todos los hechos
dpto_vic_tot = dpto_19_22.groupby(["periodo", "DEPARTAMENTO"]).apply(
    lambda x: ((x['pobre'] * x['fex_c_victi']).sum() / (x['victima'] * x['fex_c_victi']).sum() * 100))
dpto_vic_tot

periodo  DEPARTAMENTO   
2019     Amazonas           28.577279
         Antioquia          21.494652
         Arauca             23.228410
         Atlántico          14.074379
         Bogotá             12.403749
                              ...    
2022     Sucre              29.860493
         Tolima             12.498105
         Valle del Cauca    16.678370
         Vaupés             25.660977
         Vichada            52.795148
Length: 132, dtype: float64

In [224]:
# ipm por regiones. Víctimas desplazamiento
dpto_vic_des = dpto_19_22.groupby(["periodo", "DEPARTAMENTO"]).apply(
    lambda x: ((x['pobre'] * x['fex_c_victi']).sum() / (x['es_desplazado'] * x['fex_c_victi']).sum() * 100))
dpto_vic_des

periodo  DEPARTAMENTO   
2019     Amazonas           30.145604
         Antioquia          24.329313
         Arauca             24.830980
         Atlántico          14.923832
         Bogotá             13.599601
                              ...    
2022     Sucre              30.394196
         Tolima             13.103182
         Valle del Cauca    18.257646
         Vaupés             28.925578
         Vichada            55.528837
Length: 132, dtype: float64

In [226]:
## exportar resultados de regiones para víctimas 
salida2 = p1 + "ipm_departamentos.xlsx"
writer = pd.ExcelWriter(salida2)

dpto_nal.to_excel(writer, sheet_name = 'ipm_nal_dpto', header = True, index=True)
dpto_nal_nv.to_excel(writer, sheet_name = 'ipm_nal_novic_dpto', header = True, index=True)
dpto_vic_tot.to_excel(writer, sheet_name = 'ipm_vic_tot_dpto', header = True, index=True)
dpto_vic_des.to_excel(writer, sheet_name = 'ipm_vic_des_dpto', header = True, index=True)

writer.close()

In [ ]:

#Cruce vivienda hogares
# ipm_22_vh_merge = pd.merge(left=ipm_22_h, right=ipm_22_viv,
#                            left_on=['DIRECTORIO', 'SECUENCIA_ENCUESTA', 'SECUENCIA_P'], 
#                         right_on=['DIRECTORIO', 'SECUENCIA_ENCUESTA', 'SECUENCIA_P'],
#                            how='left',
#                            sort=False, validate='m:1', 
#                         suffixes=("_h", "_viv"), indicator = 'indicator_column')

# #cruce vivienda_hogares con personas
# ipm_22_vhper = pd.merge(left=ipm_22_p, right=ipm_22_vh_merge, left_on=['DIRECTORIO', "SECUENCIA_P"], 
#                         right_on=['DIRECTORIO', "SECUENCIA_ENCUESTA"], how='left', 
#                         sort=False, validate='m:1', 
#                         suffixes=("_per", "_ph"), indicator = 'indicator_column_2')

# #cruce vivienda_hogares_personas con víctimas
# ipm_22_vhper_vic = pd.merge(left=ipm_22_vhper, right=ipm_22_v, 
#                             left_on=['DIRECTORIO', 'SECUENCIA_P_per', 'ORDEN'], 
#                             right_on=['DIRECTORIO', 'SECUENCIA_P', 'ORDEN'], 
#                             how='outer',
#                             sort=False, 
#                             validate='1:1', 
#                             suffixes=("_vhper", "_vic"), 
#                             indicator = 'indicator_column_3')
##rellenar variable víctima
# def vic(df):
#     if df["VICTIMA"]==1:
#         x = 1
#     else:
#         x= 0
#     return(x)

# ##crear variable no víctima
# ipm_22_vhper_vic["no_victima"] = ipm_22_vhper_vic.apply(novic, axis = 1)

In [ ]:
### departamentos

#----------
#cruce vivienda_hogares
# dpto_vh22 = pd.merge(left=h22, right=viv22, 
#                      left_on=["DIRECTORIO", "SECUENCIA_ENCUESTA"], 
#                      right_on=["DIRECTORIO", "SECUENCIA_ENCUESTA"], 
#                      how="left",sort=False, validate="m:1", 
#                      suffixes=("_h", "_viv"), 
#                      indicator = "indicator_column")
# #cruce vivienda_hogares con personas
# dpto_vhper22 = pd.merge(left=per22, right=dpto_vh22, 
#                         left_on=["DIRECTORIO", "SECUENCIA_P"], 
#                         right_on=["DIRECTORIO", "SECUENCIA_ENCUESTA"], 
#                         how="left", sort=False, 
#                         validate="m:1", suffixes=("_per", "_ph"), 
#                         indicator = "indicator_column_2")
# #cruce vivienda_hogares_personas con víctimas
# dpto_vhper_vic22 = pd.merge(left=dpto_vhper22, 
#                             right=vic22, left_on=["DIRECTORIO", "SECUENCIA_P", "ORDEN"], 
#                             right_on=["DIRECTORIO", "SECUENCIA_P","ORDEN"], how="left",
#                             sort=False, validate="1:1", suffixes=("_vhper", "_vic"), 
#                             indicator = "indicator_column_3")